## Задание

1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).
2. Проведите предобработку:
 - Приведите тексты к нижнему регистру (text.lower()).
 - Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. 
 - Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
 - Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
 - Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
 - Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.
4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.



Загрузим данные, посмотрим на них

In [1]:
# coding=utf-8
import pandas
dataTrain = pandas.read_csv('salary-train.csv')

In [2]:
dataTrain.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


Так как предобработку нужно будет проводить не один раз, напишем функцию изменения регистра и замены пробелов

In [3]:
def transformText(inputText):
    inputText = inputText.map(lambda t: t.lower())
    inputText = inputText.replace('[^a-zA-Z0-9]', ' ', regex=True)
    return inputText

Применим TfidfVectorizer для преобразования текстов в векторы признаков

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(transformText(dataTrain['FullDescription']))

Заменим пропуски в столбцах LocationNormalized и ContractTime на 'nan'

In [5]:
dataTrain['LocationNormalized'].fillna('nan', inplace=True)
dataTrain['ContractTime'].fillna('nan', inplace=True)

Проведем one-hot-кодирование признаков

In [6]:
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_cat = enc.fit_transform(dataTrain[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объеденим все признаки в одну матрицу

In [7]:
from scipy.sparse import hstack

X_train = hstack([X_train_text, X_train_cat])

Обучим гребневую регресию

In [8]:
from sklearn.linear_model import Ridge

y_train = dataTrain['SalaryNormalized']
model = Ridge(alpha=1)
model.fit(X_train, y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

Для построения прогнозов подготовим данные

In [9]:
dataTest = pandas.read_csv('salary-test-mini.csv')

In [10]:
dataTest.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [12]:
X_test_text = vec.transform(transformText(dataTest['FullDescription']))
X_test_cat = enc.transform(dataTest[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([X_test_text, X_test_cat])

Получим прогнозы

In [13]:
model.predict(X_test)

array([56565.41755857, 37184.64695612])